# Importing Data 

Once the data is successfuly downloaded from [https://gea.esac.esa.int/archive/](https://gea.esac.esa.int/archive/), and correctly decompressed following the instroctions provided in [Neeed to add link to README from personal github](add-link), we can now import the data on Python and store it as a data frame using the package [pandas](https://pandas.pydata.org/).

Optional, add the following to the sql code to remove impurity in data:
```sql
WHERE gs.parallax < 5 * gs.parallax_error  -- Remove potential stars
AND gs.ruwe < 1.4  -- Ensure good astrometric quality
AND gs.phot_g_mean_mag < 21  -- Bright enough for good measurements
```

In [1]:
from src.data import data_download # Run Python file to download data

INFO: Query finished. [astroquery.utils.tap.core]


In [132]:
import pandas as pd
# Load into Pandas
df = pd.read_csv("csv_files/qso_full_data.csv")
print(df.head())  # Check the data

        source_id         ra       dec      pmra     pmdec  parallax  \
0   3470333738112  45.075505  0.152316 -1.072371 -3.191011  0.366321   
1   5944234902272  44.884761  0.164806 -0.121274  0.725026 -0.395659   
2   6459630980096  44.910498  0.189649  0.217806 -0.316007 -0.626561   
3   9517648372480  45.254655  0.228999 -0.552941 -1.895446 -0.917219   
4  10892037246720  45.188575  0.282424 -0.098037 -0.120580  0.001630   

   parallax_error      ruwe  phot_g_mean_mag  nu_eff_used_in_astrometry  \
0        0.901633  0.889714        20.571114                   1.526179   
1        1.340139  1.087911        20.704517                   1.647388   
2        0.548536  1.020956        20.173105                        NaN   
3        1.507964  1.031971        20.634562                        NaN   
4        0.246332  0.974657        18.787239                   1.565118   

   pmra_error  pmdec_error  pmra_pmdec_corr  astrometric_params_solved  
0    1.552507     1.226259         0.207769

### Column Content

Before we procede, we need to understand what each column contains, to do so let us print the heading of each column.

In [3]:
# Check column names
print(df.columns)

Index(['source_id', 'ra', 'dec', 'pmra', 'pmdec', 'parallax', 'parallax_error',
       'ruwe', 'phot_g_mean_mag', 'nu_eff_used_in_astrometry', 'pmra_error',
       'pmdec_error', 'pmra_pmdec_corr', 'astrometric_params_solved'],
      dtype='object')


The output above, displays:

1. <b>source_id</b>, Unique Gaia identifier for the object
2. <b>ra</b>, Right Ascension (celestial longitude) in degrees
3. <b>dec</b>, Declination (celestial latitude) in degrees
4. <b>pmra</b>, Proper motion in Right Ascension (mas/yr)
5. <b>pmdec</b>, Proper motion in Declination (mas/yr)
6. <b>parallax</b>, Parallax measurament (this is expected to be near zero for distant QSOs)
7. <b>ruwe</b>, Renormalised Unit Weight Error (this indicates the quality of the data point)
8. <b>phot_g_mean_mag</b>, Mean magnitude in Gaia's G-band 
9. <b>nu_eff_used_in_astrometry</b>, Efficient wavenumber denoted as $\nu_{eff}$, this is used to charecterise the color of a celestial object by describing how its light is distributed across different wavelenghts.
10. <b>parallax_error</b>, measure the uncentainty on parralax (standard deviation).
11. <b>pmra_error</b>, Uncertainty in pmra.
12. <b>pmdec_error</b>, Uncertainty in pmdec_error.

We want to make suere the data matches the description provided in the papaer, i.e. we expect there to be 1614173 sources which are identified as QSO-like objects. 

# Filtering Data
In this section we aim to remove potential outliers from the five parameter solution dataset.

## Plan 

1. Initial Least Square Fit $\rightarrow$ Fit the VSH model using all data.
2. Compute $X^2$ Values $\rightarrow$ For each QSO
3. Compute the Median $\rightarrow$ Compute the median of all $X$ values.
4. Reject Outliers $\rightarrow$ If $X>\kappa \times \text{median}(X)$, mark the source as outlier.
5. Refit $\rightarrow$ Exclude outliers and re-run the fit.
6. Iterate $\rightarrow$ Repeate steps (e.g. 2-5) until convergence (i.e. outlier stops changing).

## Procedure In EDR3
1. Perform Least Square $\rightarrow$ estimate VSH coefficients
2. $\kappa$-Clipping $\rightarrow$ remove outliers
3. Perform Bootstrapping $\rightarrow$ quantify the uncertainty of results

## Our approach
Keep step 1 and 2 from EDR3, nut instrad:
1. Perform Least Square $+$ $\kappa$-Clipping $\rightarrow$ filter the data
2. Perform HMC (Bayesian inference) sampling on filtered dataset $\rightarrow$ achieving posterior samples, with VSH coefficient estimate and uncertainities.

In [3]:
import jax.numpy as jnp
from jax import jit, vmap
from functools import partial, lru_cache
from src.models.vsh_model import*
import pandas as pd
from iminuit import Minuit # to perform least square
from src.models.configuration import*
from src.data.data_utils import*
import gc

In [4]:
# Load data
df = load_qso_dataframe()
angles, obs, error = config_data(df)

In [5]:
def robust_least_squares_fit(angles, obs, error, theta_init, lmax = 3, kappa=3.0, max_iter=10):

    
    alpha, delta = angles
    mu_a_obs, mu_d_obs = obs
    s_mu_a, s_mu_d, rho = error

    keep = jnp.ones_like(alpha, dtype=bool)
    theta = theta_init

    prev_outliers = None

    for iteration in range(max_iter):
        print('Iteration:', iteration+1)
        alpha_k, delta_k = alpha[keep], delta[keep]
        obs_k = (mu_a_obs[keep], mu_d_obs[keep])
        err_k = (s_mu_a[keep], s_mu_d[keep], rho[keep])
        angles_k = (alpha_k, delta_k)

        def least_square_wrapper(*theta_flat):
            theta_arr = jnp.array(theta_flat)
            return least_square(angles_k, obs_k, err_k, theta_arr, lmax=lmax, grid=False)

        m = Minuit(least_square_wrapper, *theta)
        m.errordef = Minuit.LEAST_SQUARES

        m.migrad()

        theta = jnp.array([m.values[name] for name in m.parameters])

        C0 = 1000/np.sqrt(8*np.pi/3)
        C1 = 1000/np.sqrt(4*np.pi/3)

        print(f'Current g components [μas/yr]: gx = {-theta[4]*C1}, gy = {theta[5]*C1}, gz = {theta[1]*C0}')

        del m
        gc.collect()
        jax.clear_caches()

        # Compute X^2 over full dataset (not just kept subset)
        X = np.sqrt(compute_X2(alpha, delta, mu_a_obs, mu_d_obs, s_mu_a, s_mu_d, rho, theta, lmax))
        median_X = jnp.median(X)
        keep = X < kappa*median_X

        print(f"Rejected: {(~keep).sum()} sources")

        if prev_outliers is not None and jnp.array_equal(keep, prev_outliers):
            print(f"Converged after {iteration+1} iterations.")
            break
        prev_outliers = keep

    return theta, keep


In [4]:
lmax = 3
total_params = count_vsh_coeffs(lmax)
theta_init = jnp.zeros(total_params)

theta, keep = robust_least_squares_fit(angles, obs, error, theta_init)

print(f'Length of keep array: {len(keep)}')

Iteration: 1
Current g components [μas/yr]: gx = -0.293548583984375, gy = -4.0919294357299805, gz = -2.8633511066436768
Rejected: 3787 sources
Iteration: 2
Current g components [μas/yr]: gx = -0.1732703298330307, gy = -4.25071382522583, gz = -2.678118944168091
Rejected: 3788 sources
Iteration: 3
Current g components [μas/yr]: gx = -0.16842861473560333, gy = -4.266857147216797, gz = -2.665255308151245
Rejected: 3786 sources
Iteration: 4
Current g components [μas/yr]: gx = -0.16842861473560333, gy = -4.266857147216797, gz = -2.665255308151245
Rejected: 3787 sources
Iteration: 5
Current g components [μas/yr]: gx = -0.13661928474903107, gy = -4.272395610809326, gz = -2.6614580154418945
Rejected: 3785 sources
Iteration: 6
Current g components [μas/yr]: gx = -0.13488395512104034, gy = -4.272395610809326, gz = -2.6614580154418945
Rejected: 3785 sources
Converged after 6 iterations.
Length of keep array: 1215942


In [ ]:
df_clean = df.loc[np.array(keep)]
# Saving filtered dataset
df_clean.to_csv('csv_files/filtered_qso_data.csv', index=False)

# Model

## Least Square (for $l=1$ Toy Model)

We successfully downloaded, loaded and filtered the data for 5 parameters solution, we defined the toroidal and spheroidal functions ($T_{lm}$ and $S_{lm}$), respectively modelling the right ascension (ra) and declination (dec). For $\alpha \in [0, 2\pi]$ and $\delta \in [-\pi/2,\pi/2]$ we visualised the VSH vector fields. We now want to perform a MLE on the dataset.

To do so, we will follow closely the procedure presented in the main paper (Gaia Early Data Release 3 Acceleration of the Solar System from Gaia astrometry). This assumes that the noise follows a Gaussian model, i.e. the astrometric measurement errors (in proper motion, parallax, etc.) are:
- Unbiased (zero mean),
- Independent between different sources (quasars),
- With known standard deviation and correlations, as provided in Gaia EDR3.

This allows the least-square estimation framework and the statistical significance tests, in particular using the $\chi^2$ distributions for assessing power in VSH.

Recall Eq. 5 amd 7:
$$
V(\alpha, \delta) = \sum_{l=1}^{l_{\text{max}}} \left( t_{l0} T_{l0} + s_{l0} S_{l0}
+ 2 \sum_{m=1}^{l} \left( t_{lm}^{\mathbb{R}} T_{lm}^{\mathbb{R}} - t_{lm}^{\mathbb{I}} T_{lm}^{\mathbb{I}} + s_{lm}^{\mathbb{R}} S_{lm}^{\mathbb{R}} - s_{lm}^{\mathbb{I}} S_{lm}^{\mathbb{I}} \right) \right)\tag{5}
$$

$$
X^2 = \begin{bmatrix}
\Delta\mu_{\alpha^*} & \Delta\mu_{\delta} \\
\end{bmatrix}
\begin{bmatrix}
\sigma_{\mu_{\alpha^*}}^2 & \rho_{\mu}\sigma_{\mu_{\alpha^*}}\sigma_{\mu_{\delta}} \\
\rho_{\mu}\sigma_{\mu_{\alpha^*}}\sigma_{\mu_{\delta}} & \sigma_{\mu_{\delta}}^2
\end{bmatrix}
\begin{bmatrix}
\Delta\mu_{\alpha^*} \\ \Delta\mu_{\delta} 
\end{bmatrix}\tag{7}
$$

where:
- $\Delta\mu_{\alpha^*} = \mu_{\alpha^* \text{obs}} - V_{\alpha^* \text{model}}$ is the difference between observed and predicted proper motion right ascension (ra).
- $\Delta\mu_{\delta} = \mu_{\delta \text{obs}} - V_{\delta \text{model}}$ is the difference between observed and predicted proper motion declination (dec).

Since each proper motion componet is assumed to follow a Gaussian distribution, MLE simplifies to a weighted least squares. Hence our objective is to minimise Eq. 7:

$$
\sum_k \begin{bmatrix}
\Delta\mu_{\alpha^*} & \Delta\mu_{\delta} \\
\end{bmatrix}
\begin{bmatrix}
\sigma_{\mu_{\alpha^*}}^2 & \rho_{\mu}\sigma_{\mu_{\alpha^*}}\sigma_{\mu_{\delta}} \\
\rho_{\mu}\sigma_{\mu_{\alpha^*}}\sigma_{\mu_{\delta}} & \sigma_{\mu_{\delta}}^2
\end{bmatrix}
\begin{bmatrix}
\Delta\mu_{\alpha^*} \\ \Delta\mu_{\delta} 
\end{bmatrix}
$$

Below we are generating synthetic data to test our least square function and the coded VSH model (the VSH expansion Eq.5). We set $l_{max}=2$ when generating data.

Generate synthetic data.

In [ ]:
from src.data import generate_synthetic_data
generate_synthetic_data # Generate synthetic data

Load synthetic data and true coefficient values.

In [6]:
import pandas as pd
import numpy as np

df_synthetic = pd.read_csv('synthetic_data/synthetic_vsh_data.csv') # loead synthetic 
true_coeff = np.load('synthetic_data/theta_true.npy') # loead true VSH coefficients
angles_gen, obs_gen, error_gen = config_data(df_synthetic) # configurate the data for input

In [7]:
from iminuit import Minuit # to perform least square
from src.models.configuration import*
from src.data.data_utils import*
from src.models.vsh_model import*
import jax.numpy as jnp
from jax import jit, vmap
from functools import partial, lru_cache

# Test Toy Model \& Model on Synthetic Data

- First by fitting the least square (see function `toy_least_square` in [`src.models.vsh_model.py`](src/models/vsh_model.py)) with `iminuit`.
- Additionaly testing both least square fit and HMC on universal vsh model and least square fucntion (respectivelly `model_vsh` and `least_square` and [`src.models.vsh_model.py`](src/models/vsh_model.py)).

Note toy model only work for $l_{max}$ = 1, recall synthetic data was generated with $l=2$!

In [14]:
# Bind fixed arguments into a new function
bound_least_square = partial(toy_least_square, angles_gen, obs_gen, error_gen) 

# Now Minuit only sees the 6 free parameters
m = Minuit(bound_least_square,
           t_10=0.0, t_11r=0.0, t_11i=0.0,
           s_10=0.0, s_11r=0.0, s_11i=0.0)

m.errordef=Minuit.LEAST_SQUARES

m.migrad()

print('Toy Model Result l = 1:')
theta_fit = jnp.array([m.values[k] for k in m.parameters])
print("Fitted parameters values:")
print(theta_fit)
print("True values:")
print(true_coeff[:count_vsh_coeffs(1)])

Toy Model Result l = 1:
Fitted parameters values:
[ 0.03217079  0.01450661  0.0113407   0.1311283   0.02000312 -0.06413274]
True values:
[ 0.05372004  0.05742959 -0.02012502 -0.00375978  0.00838665 -0.04013964]


Least square with $l=1$

In [15]:
lmax = 1
total_params = count_vsh_coeffs(lmax) 

# Flat vector theta: [t10, ..., t_lmaxm, s10, ..., s_lmaxm]
theta_init = jnp.zeros(total_params)

# Fix everything except theta
def least_square_wrapper(*theta_flat):
    theta = jnp.array(theta_flat)  # reconstructs the vector from scalars
    return least_square(angles_gen, obs_gen, error_gen, theta, lmax=lmax, grid=False)

m = Minuit(least_square_wrapper, *theta_init)

m.errordef = Minuit.LEAST_SQUARES

m.migrad()

print('Compleate least square result l = 1:')
theta_fit = jnp.array([m.values[k] for k in m.parameters])
print("Fitted parameters values:")
print(theta_fit)
print("True values:")
print(true_coeff[:count_vsh_coeffs(1)])

Compleate least square result l = 1:
Fitted parameters values:
[ 0.03216825  0.13111432  0.01450038  0.01133812  0.02000075 -0.0641298 ]
True values:
[ 0.05372004  0.05742959 -0.02012502 -0.00375978  0.00838665 -0.04013964]


Least Square for $l=2$

In [16]:
lmax = 2
total_params = count_vsh_coeffs(lmax) 

# Flat vector theta: [t10, ..., t_lmaxm, s10, ..., s_lmaxm]
theta_init = jnp.zeros(total_params)

# Fix everything except theta
def least_square_wrapper(*theta_flat):
    theta = jnp.array(theta_flat)  # reconstructs the vector from scalars
    return least_square(angles_gen, obs_gen, error_gen, theta, lmax=lmax, grid=False)

m = Minuit(least_square_wrapper, *theta_init)

m.errordef = Minuit.LEAST_SQUARES

m.migrad()

print('Compleate least square result l = 2:')
theta_fit = jnp.array([m.values[k] for k in m.parameters])
print("Fitted parameters values:")
print(theta_fit)
print("True values:")
print(true_coeff[:count_vsh_coeffs(2)])

Compleate least square result l = 2:
Fitted parameters values:
[ 0.05237787  0.05555838 -0.02015737 -0.00370113  0.01000114 -0.0395992
 -0.01779413  0.02249806  0.03067211 -0.03782388  0.05861543 -0.05849521
  0.01581841  0.0072174   0.0466077   0.05248069]
True values:
[ 0.05372004  0.05742959 -0.02012502 -0.00375978  0.00838665 -0.04013964
 -0.02277664  0.02273766  0.02961199 -0.03947825  0.05824246 -0.05696608
  0.01680502  0.0075229   0.04790566  0.0521445 ]


Least square $l=3$ 

In [17]:
lmax = 3
total_params = count_vsh_coeffs(lmax) 

# Flat vector theta: [t10, ..., t_lmaxm, s10, ..., s_lmaxm]
theta_init = jnp.zeros(total_params)

# Fix everything except theta
def least_square_wrapper(*theta_flat):
    theta = jnp.array(theta_flat)  # reconstructs the vector from scalars
    return least_square(angles_gen, obs_gen, error_gen, theta, lmax=lmax, grid=False)

m = Minuit(least_square_wrapper, *theta_init)

m.errordef = Minuit.LEAST_SQUARES

m.migrad()

print('Compleate least square result l = 2:')
theta_fit = jnp.array([m.values[k] for k in m.parameters])
print("Fitted parameters values:")
print(theta_fit[:count_vsh_coeffs(2)])
print("True values:")
print(true_coeff[:count_vsh_coeffs(2)])

Compleate least square result l = 2:
Fitted parameters values:
[ 0.03014502  0.03355306 -0.0007573   0.01433174  0.01033522 -0.01194067
 -0.01777831  0.00241366  0.02045461 -0.04025296  0.04289113 -0.06602865
  0.00887562  0.01604122  0.03837589  0.01503773]
True values:
[ 0.05372004  0.05742959 -0.02012502 -0.00375978  0.00838665 -0.04013964
 -0.02277664  0.02273766  0.02961199 -0.03947825  0.05824246 -0.05696608
  0.01680502  0.0075229   0.04790566  0.0521445 ]


# Test HMC Sampling on Synthetic Data

In [ ]:
import jax
from numpyro.infer import NUTS, MCMC
from numpyro.diagnostics import summary
import numpyro

In [20]:
def chi2_jit(angles, obs, error, theta, lmax):
    return least_square(angles, obs, error, theta, lmax=lmax, grid=False)
chi2_jit = jit(chi2_jit, static_argnames=['lmax'])

# Define Model with Uniform prior
def model_w_uni_prior(angles, obs, error, limit = 0.1, lmax = 3):
    total_params = count_vsh_coeffs(lmax)
    # Prior on all VSH coefficients (both toroidal and spheroidal)
    theta = numpyro.sample("theta", dist.Uniform(-limit, limit).expand([total_params]))
    # Least-squares residuals: we assume Gaussian-distributed residuals
    chi2_val = chi2_jit(angles, obs, error, theta, lmax=lmax)

    # The log-likelihood is proportional to -0.5*chi^2
    numpyro.factor("likelihood", -0.5*chi2_val)

# Definie Model with Gaussian Prior
def model_w_norm_prior(angles, obs, error, std = 1, lmax = 3):

    total_params = count_vsh_coeffs(lmax)
    # Prior on all VSH coefficients (both toroidal and spheroidal)
    theta = numpyro.sample("theta", dist.Normal(0., std).expand([total_params]))
    # Least-squares residuals: we assume Gaussian-distributed residuals
    chi2_val = chi2_jit(angles, obs, error, theta, lmax=lmax)

    # The log-likelihood is proportional to -0.5*chi^2
    numpyro.factor("likelihood", -0.5*chi2_val)


n_s = 1000 # number of samples
n_warmup = 2000 #  number of warmups 
n_chains = 3 # numbe of chains

In [23]:
x = np.random.randint(1, 101)
rng_key = jax.random.key(x)

kernel_uni = NUTS(model_w_uni_prior, target_accept_prob=0.75) # this is to make sure acceptance does not exceed 90%
mcmc_uni = MCMC(kernel_uni, num_warmup=n_warmup, num_samples=n_s, num_chains=n_chains, chain_method='sequential', progress_bar=True)
mcmc_uni.run(rng_key, angles = angles_gen, obs = obs_gen, error = error_gen, limit = 1, lmax=3)
ps_w_uni_prior = mcmc_uni.get_samples()

diagnostics = summary(mcmc_uni.get_samples(group_by_chain=True))
divergences = mcmc_uni.get_extra_fields()["diverging"]  # shape: (num_samples * num_chains,)
num_divergences = divergences.sum()
print("Number of divergences:", num_divergences)

del mcmc_uni
gc.collect()
jax.clear_caches()

warmup:   4%|▍         | 119/3000 [00:36<14:36,  3.29it/s, 1023 steps of size 1.08e-03. acc. prob=0.70]


KeyboardInterrupt: 

In [ ]:
x = np.random.randint(1, 101)
rng_key = jax.random.key(x)

kernel_norm = NUTS(model_w_norm_prior, target_accept_prob=0.75)
mcmc_norm = MCMC(kernel_norm, num_warmup=n_warmup, num_samples=n_s, num_chains=n_chains, progress_bar=True)
mcmc_norm.run(rng_key, angles = angles_gen, obs = obs_gen, error = error_gen, std = 1, lmax=3)

ps_w_norm_prior = mcmc_norm.get_samples()

mcmc_norm.print_summary()
del mcmc_norm
gc.collect()
jax.clear_caches()